In [1]:
# -*- coding: utf-8
# Repórter Brasil (http://ruralometro.reporterbrasil.org.br/)
# Reinaldo Chaves (@paidatocandeira)
# Cria base com dívidas de deputados federais na Previdência Social e quais delas são do Funrural, baseado em informações da Procuradoria-Geral da Fazenda Nacional via LAI:  http://www.acessoainformacao.gov.br/assuntos/busca-de-pedidos-e-respostas/busca-de-pedidos-e-respostas
#

In [ ]:
import pandas as pd

Formato float

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

Carrega candidatos de 2014

In [3]:
eleitos_d = pd.read_excel('/home/reinaldo/Documentos/Code/RB2/candidatos/repo/resultados/eleitos_final_2014.xlsx',sheet_name='Sheet1', converters={'sequencial': lambda x: str(x),
                                                                                                                                                   'cpf': lambda x: str(x),  
                                                                                                                                                   'deputado_id': lambda x: str(x)})

In [4]:
eleitos_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 15 columns):
uf                   513 non-null object
cargo                470 non-null object
nome_completo        513 non-null object
sequencial           513 non-null object
cpf                  513 non-null object
nome_urna            513 non-null object
partido_eleicao      513 non-null object
situacao_urna        513 non-null object
condicao             513 non-null object
deputado_id          513 non-null object
legislatura          513 non-null int64
partido_atual        513 non-null object
situacao_mandato     513 non-null object
uri                  513 non-null object
bancada_ruralista    513 non-null object
dtypes: int64(1), object(14)
memory usage: 60.2+ KB


Carrega dívidas com o INSS de deputados

In [5]:
inss = pd.read_excel('dados/deputados_devedores_inss_maio2018.xlsx',sheet_name='Plan1')

In [6]:
inss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
CPF_socio                     100 non-null object
Nome_socio                    100 non-null object
Qualificacao_socio            100 non-null object
CPF_CNPJ_Devedor_Detalhado    100 non-null object
Nome_Devedor                  100 non-null object
Divida_Previdenciaria         100 non-null object
valor_irregular               100 non-null float64
valor_regular                 100 non-null float64
TOTAL                         100 non-null float64
FUNRURAL                      100 non-null object
dtypes: float64(3), object(7)
memory usage: 7.9+ KB


Limpa CNPJ/CPF

In [7]:
def limpeza1(linha):
    documento = str(linha["CPF_socio"])
    documento = documento.strip()
    documento = documento.replace('.', '')
    documento = documento.replace('-', '')
    documento = documento.replace('/', '')
    return str(documento)

def limpeza2(linha):
    documento = str(linha["CPF_CNPJ_Devedor_Detalhado"])
    documento = documento.strip()
    documento = documento.replace('.', '')
    documento = documento.replace('-', '')
    documento = documento.replace('/', '')
    return str(documento)

In [8]:
inss["cpf_limpo"] = inss.apply(limpeza1, axis=1)
inss['cnpj_limpo'] = inss.apply(limpeza2, axis=1)

Verifica quais devedores são candidatos 2014

In [9]:
dividas_eleitos_2014 = pd.merge(inss, eleitos_d, left_on='cpf_limpo', right_on='cpf')

In [10]:
dividas_eleitos_2014.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92 entries, 0 to 91
Data columns (total 27 columns):
CPF_socio                     92 non-null object
Nome_socio                    92 non-null object
Qualificacao_socio            92 non-null object
CPF_CNPJ_Devedor_Detalhado    92 non-null object
Nome_Devedor                  92 non-null object
Divida_Previdenciaria         92 non-null object
valor_irregular               92 non-null float64
valor_regular                 92 non-null float64
TOTAL                         92 non-null float64
FUNRURAL                      92 non-null object
cpf_limpo                     92 non-null object
cnpj_limpo                    92 non-null object
uf                            92 non-null object
cargo                         86 non-null object
nome_completo                 92 non-null object
sequencial                    92 non-null object
cpf                           92 non-null object
nome_urna                     92 non-null object
partido_elei

In [11]:
dividas_eleitos_2014.reset_index().head()

,index,CPF_socio,Nome_socio,Qualificacao_socio,CPF_CNPJ_Devedor_Detalhado,Nome_Devedor,Divida_Previdenciaria,valor_irregular,valor_regular,TOTAL,...,nome_urna,partido_eleicao,situacao_urna,condicao,deputado_id,legislatura,partido_atual,situacao_mandato,uri,bancada_ruralista
0,0,055.282.278-79,IOLANDA KEIKO MIASHIRO OTA,SOCIO ADMINISTRADOR,00.002.874/0001-88,MAGAZINE FUR LTDA,Sim,"154,205.71",0.00,"154,205.71",...,OTA,PSB,ELEITO POR QP,Titular,160659,55,PSB,Exercício,https://dadosabertos.camara.leg.br/api/v2/depu...,não
1,1,019.609.704-53,LUIZ GONZAGA PATRIOTA,PRESIDENTE,00.044.180/0001-03,CLUBE DO CONGRESSO,Sim,0.00,"1,344,120.95","1,344,120.95",...,GONZAGA PATRIOTA,PSB,ELEITO POR QP,Titular,74419,55,PSB,Exercício,https://dadosabertos.camara.leg.br/api/v2/depu...,sim
2,2,019.609.704-53,LUIZ GONZAGA PATRIOTA,SOCIO ADMINISTRADOR,70.065.941/0001-99,PERGRAN PERNAMBUCO GRANITOS INDUSTRIA COM E EX...,Sim,"90,852.91",0.00,"90,852.91",...,GONZAGA PATRIOTA,PSB,ELEITO POR QP,Titular,74419,55,PSB,Exercício,https://dadosabertos.camara.leg.br/api/v2/depu...,sim
3,3,077.324.548-04,RONALDO FONSECA DE SOUZA,PRESIDENTE,00.424.952/0001-32,IGREJA EVANGELICA ASSEMBLEIA DE DEUS DE TAGUAT...,Sim,0.00,"29,046.19","29,046.19",...,RONALDO FONSECA,PROS,ELEITO POR QP,Titular,160637,55,PODE,Licença,https://dadosabertos.camara.leg.br/api/v2/depu...,não
4,4,189.313.672-87,LUIZ GIONILSON PINHEIRO BORGES,SOCIO,00.881.907/0001-07,BEIJA FLOR RADIODIFUSAO LTDA,Sim,"132,772.94",0.00,"132,772.94",...,CABUCU,PMDB,ELEITO POR MÉDIA,Titular,105112,55,MDB,Exercício,https://dadosabertos.camara.leg.br/api/v2/depu...,não


Maiores dívidas

In [12]:
dividas_eleitos_2014.groupby("nome_urna").TOTAL.sum().reset_index().sort_values("TOTAL", ascending=False).head(10)

,nome_urna,TOTAL
37,MARINALDO ROSENDO,"114,264,293.83"
16,ELCIONE,"40,311,963.93"
1,ALFREDO KAEFER,"30,153,685.60"
42,NEWTON CARDOSO JR,"18,057,050.95"
23,GIUSEPPE VECCI,"10,881,554.36"
11,CELSO RUSSOMANNO,"2,207,515.93"
28,JOAO GUALBERTO,"1,697,012.98"
22,FELIX JR,"1,519,355.92"
24,GONZAGA PATRIOTA,"1,434,973.86"
27,JHC,"1,044,671.75"


In [13]:
dividas_eleitos_2014.to_excel('resultados/dividas_inss_apenas_eleitos_2014.xlsx',sheet_name='Sheet1')

Apenas as dívidas com Funrural

In [18]:
funrural = dividas_eleitos_2014[(dividas_eleitos_2014['FUNRURAL'] == '744 - FUNRURAL')]

In [19]:
funrural.groupby("nome_urna").TOTAL.sum().reset_index().sort_values("TOTAL", ascending=False)

,nome_urna,TOTAL
1,NEWTON CARDOSO JR,"1,827,665.29"
0,DILCEU SPERAFICO,"68,845.88"
